In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pickle

with open('drive/My Drive/Machine_Learning-prj/vietnamese_address/images_64_1595.pkl', 'rb') as f:
    images = pickle.load(f)
    
with open('drive/My Drive/Machine_Learning-prj/vietnamese_address/scripts_64_1595.pkl', 'rb') as f:
    scripts = pickle.load(f)

In [0]:
# import pickle

# with open('drive/My Drive/Machine_Learning-prj/vietnamese_address/images_address.pkl', 'rb') as f:
#     images = pickle.load(f)
    
# with open('drive/My Drive/Machine_Learning-prj/vietnamese_address/scripts_address.pkl', 'rb') as f:
#     scripts = pickle.load(f)

In [4]:
print(len(images), len(scripts))
images = images[:7200]
scripts = scripts[:7200]
len(images), len(scripts)

7260 7260


(7200, 7200)

In [0]:
from PIL import Image
import numpy as np


def scale(img, scale_prob = 0.5, scale_stdv = 0.12):
    """
    It scale the image provided to a random size chosen from the sample of a lognormal probability
    function with scale_stdv standard deviation. The probabilite that the image will be scaled is scale_prob.
    
    Args:
        img: numpy format image.
        scale_prob: probability of scaling the image.
        scale_stdv: standard deviation of the lognormal probability function used to chose
            the size of the scaled image.
            
    Returns:
        img: the scaled (or not) image.
    
    """
    scale = np.random.binomial(1, scale_prob)
    
    if scale:
        
        imgPIL = Image.fromarray(img)
        ho, vo = imgPIL.size
        scale_factor = np.random.lognormal(sigma=scale_stdv)
        hn, vn = int(scale_factor*ho), int(scale_factor*vo)
        img_sc = imgPIL.resize((hn, vn))
        
        img = np.array(img_sc).reshape((vn,hn))
                
        if hn > ho:
            img = img[int(vn/2)-int(vo/2):int(vn/2)+int(np.ceil(vo/2)) ,int(hn/2)-int(ho/2):int(hn/2)+int(np.ceil(ho/2))] 
        else:
            img = np.pad(img, ((int((vo-vn)/2), int(np.ceil((vo-vn)/2))),((int((ho-hn)/2), int(np.ceil((ho-hn)/2))))), mode='constant')
            
    return img

def shear(img, shear_prob = 0.5, shear_prec = 4):
    """
    It shear the image provided to a random angle chosen from the sample of a vonmises probability
    function with shear_prec kappa parameter. The probability that the image will be sheared is shear_prob.
    
    Args:
        img: numpy format image.
        shear_prob: probability of shearing the image.
        shear_prec: kappa parameter of the vonmises probability function used to chose
            the angle to shear image.
            
    Returns:
        img: the sheared (or not) image.
    
    """
    shear = np.random.binomial(1, shear_prob)

    if shear:
        import cv2
        rows,cols = img.shape
        shear_angle = np.random.vonmises(0, kappa = shear_prec)
        m = np.tan(shear_angle)
        
        pts1 = np.float32([[50,50],[200,50],[50,200]])
        pts2 = np.float32([[50,50],[200,50],[50+m*150,200]])
        M = cv2.getAffineTransform(pts1,pts2)
            
        img = cv2.warpAffine(img,M,(cols,rows))
        
    return img

def rotate(img, rotate_prob = 0.5, rotate_prec = 100):
    """
    It rotate the image provided to a random angle chosen from the sample of a vonmises probability
    function with rotate_prec kappa parameter. The probability that the image will be sheared is rotate_prob.
    
    Args:
        img: numpy format image.
        rotate_prob: probability of rotating the image.
        rotate_prec: kappa parameter of the vonmises probability function used to chose
            the angle to rotate image.
            
    Returns:
        img: the rotated (or not) image.
    
    """    
    rotate = np.random.binomial(1, rotate_prob)
    
    if rotate:
        import cv2
        rows,cols = img.shape
        rotate_prec = rotate_prec * max(rows/cols, cols/rows)
        rotate_angle = np.random.vonmises(0, kappa = rotate_prec)*180/np.pi
        M = cv2.getRotationMatrix2D((cols/2,rows/2),rotate_angle,1)
        img = cv2.warpAffine(img,M,(cols,rows))
    
    return img

def translate(img, translate_prob = 0.5, translate_stdv = 0.02):
    """
    It translate the image provided to a random position chosen from the sample of a normal probability
    function with translate_stdv standard deviation. The probability that the image will be translated is tanslate_prob.
    
    Args:
        img: numpy format image.
        translate_prob: probability of translating the image.
        translate_stdv: standard deviation of the normal probability function used to chose
            the position to translate the image.
            
    Returns:
        img: the translated (or not) image.
    
    """
    translate = np.random.binomial(1, translate_prob)
    
    if translate:
        import cv2
        rows,cols = img.shape
        h_translation_factor = np.random.normal(0, scale = translate_stdv * cols)
        v_translation_factor = np.random.normal(0, scale = translate_stdv * rows)
        M = np.float32([[1,0,h_translation_factor],[0,1,v_translation_factor]])
        img = cv2.warpAffine(img,M,(cols,rows))
    
    return img

def dilate(img, dilation_prob = 0.5 , dilation_srate = 0.4, dilation_rrate = 1):
    
    dilate = np.random.binomial(1, dilation_prob)
    
    if dilate:
        import cv2
        kernel_size = np.min([2*np.random.geometric(dilation_srate)+1, 15])
        kernel = np.zeros([kernel_size, kernel_size])
        center = np.array([int(kernel_size/2), int(kernel_size/2)])
        for x in range(kernel_size):
            for y in range(kernel_size):
                d = np.linalg.norm(np.array([x,y])-center)
                p = np.exp(-d*1)
                value = np.random.binomial(1, p)
                kernel[x,y] = value or 10**-16
        
        img = cv2.dilate(img,kernel,iterations = 1)
    
    return img

def erode(img, erosion_prob = 0.5 , erosion_srate = 0.8, erosion_rrate = 1.2):
    
    erode = np.random.binomial(1, erosion_prob)
    
    if erode:
        import cv2
        kernel_size = np.min([2*np.random.geometric(erosion_srate)+1, 15])
        kernel = np.zeros([kernel_size, kernel_size])
        center = np.array([int(kernel_size/2), int(kernel_size/2)])
        for x in range(kernel_size):
            for y in range(kernel_size):
                d = np.linalg.norm(np.array([x,y])-center)
                p = np.exp(-d*1)
                value = np.random.binomial(1, p)
                kernel[x,y] = value or 10**-16
        
        img = cv2.erode(img,kernel,iterations = 1)
    
    return img

def distort(img_list):
    new_list=[]
    for ind, img_np in enumerate(img_list):
        #img_np = 255 - img_np
        img_np = translate(img_np)
        img_np = rotate(img_np)
        img_np = shear(img_np)
        img_np = scale(img_np)
        img_np = dilate(img_np)
        img_np = erode(img_np)
        new_list.append(img_np)
    
    return new_list

In [6]:
from tqdm import tqdm

images_augumented = []
scripts_augumented = []

for i, img in tqdm(enumerate(images)):
    images_augumented.append(img)
    scripts_augumented.append(scripts[i])
    
    for k in range(3):
        img_tmp = translate(img)
        img_tmp = rotate(img_tmp)
        img_tmp = shear(img_tmp)
        img_tmp = scale(img_tmp)
        img_tmp = dilate(img_tmp)
        img_tmp = erode(img_tmp)
        images_augumented.append(img_tmp)
        scripts_augumented.append(scripts[i])

7200it [00:36, 195.97it/s]


In [7]:
len(images_augumented),  len(scripts_augumented)

(28800, 28800)

In [0]:
del images
del scripts

In [0]:
alphabet = u'\ !%"#&\'()*+,-./0123456789:;?AÁẢÀÃẠÂẤẨẦẪẬĂẮẲẰẴẶBCDĐEÉẺÈẼẸÊẾỂỀỄỆFGHIÍỈÌĨỊJKLMNOÓỎÒÕỌÔỐỔỒỖỘƠỚỞỜỠỢPQRSTUÚỦÙŨỤƯỨỬỪỮỰVWXYÝỶỲỸỴZaáảàãạâấẩầẫậăắẳằẵặbcdđeéẻèẽẹêếểềễệfghiíỉìĩịjklmnoóỏòõọôốổồỗộơớởờỡợpqrstuúủùũụưứửừữựvwxyýỷỳỹỵz'

def text_to_labels(text):
    ret = []
    for char in text:
        ret.append(alphabet.find(char))
    return ret


# Reverse translation of numerical classes back to characters
def labels_to_text(labels):
    ret = []
    for c in labels:
        if c == len(alphabet):  # CTC Blank
            ret.append("")
        else:
            ret.append(alphabet[c])
    return "".join(ret)

In [10]:
len(alphabet)

215

In [0]:
import numpy as np

x_train_len = np.ones((len(images_augumented), 1), dtype=np.float32) * 254
y_train_len = np.asarray([len(scripts_augumented[i]) for i in range(len(scripts_augumented))])

In [0]:
labels_script = [text_to_labels(s) for s in scripts_augumented]

In [13]:
from keras.preprocessing import sequence

# images = sequence.pad_sequences(images, value=255, dtype='uint8',
                                         #padding="post", truncating='post')
labels_script = sequence.pad_sequences(labels_script, value=len(alphabet), maxlen=256,
                                         dtype="float32", padding="post")

Using TensorFlow backend.


In [14]:
y_train_len.shape

(28800,)

In [0]:
#Building model
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Activation, GRU, Dense, Input, Add, Concatenate, Reshape, Lambda
from keras.optimizers import SGD, RMSprop
from keras import backend as K
import keras
import cv2

In [0]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [0]:
def preprocessing_image(image):
    image = image/255.
    image = cv2.copyMakeBorder(image, 0, 0, 0, 1595 - image.shape[1] ,cv2.BORDER_CONSTANT,value=0)
    image = np.reshape(image, (64, 1595, 1))
    image = image.transpose((1, 0, 2)) 
    return image

In [0]:
def image_batch_generator(images, labels, x_train_len, y_train_len, batch_size):
    while True:
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = len(images), size = batch_size)
        input = []
        output = []
        input_len = []
        output_len = []
        
        # Read in each input, perform preprocessing and get labels
        for i in batch_paths:
            input.append(preprocessing_image(image=images[i]))
            output.append(labels[i])
            input_len.append(x_train_len[i])
            output_len.append(y_train_len[i])
        input = np.array(input)
        output = np.array(output)
        input_len = np.array(input_len)
        output_len = np.array(output_len)
    
        # Return a tuple of (input,output) to feed the network
        
        yield( [input, output, input_len, output_len], np.zeros(len(input)) )

In [19]:
img_w = 1595
# Input Parameters
img_h = 64
# Network parameters
conv_filters = 16
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 256
rnn_size = 256
minibatch_size = 32
unique_tokens = 216

 
input_shape = (img_w, img_h, 1)

act = 'relu'
input_data = Input(name='the_input', shape=input_shape, dtype='float32')

inner = Conv2D(64, 3, padding='same', activation=act,name='conv1', kernel_initializer='he_normal')(input_data)
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)
inner = Conv2D(128, 3, padding='same',activation=act, name='conv2', kernel_initializer='he_normal')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Conv2D(256, 3, padding='same',activation=act, name='conv3', kernel_initializer='he_normal')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max3')(inner)

conv_to_rnn_dims = (256, 199*8)
inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

# cuts down input size going into RNN:
inner = Dense(time_dense_size, activation=act, kernel_initializer='he_normal', name='dense1')(inner)

# Two layers of bidirectional GRUs
# GRU seems to work as well, if not better than LSTM:
gru_1 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
gru1_merged = Add()([gru_1, gru_1b])
gru_2 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', go_backwards=True, name='gru2_b')(gru1_merged)

# transforms RNN output to character activations:
inner = Dense(unique_tokens,name='dense2')(Concatenate()([gru_2, gru_2b]))
y_pred = Activation('softmax', name='softmax')(inner)

# #Get text predict
# out = K.get_value(K.ctc_decode(y_pred, input_length=np.ones(y_pred.shape[0])*y_pred.shape[1],
#                          greedy=True)[0][0])

Model(inputs=input_data, outputs=y_pred).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 1595, 64, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 1595, 64, 64) 640         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 797, 32, 64)  0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 797, 32, 128) 73856       max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [0]:
labels = Input(name='the_labels', shape=[256, ], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
# Keras doesn't currently support loss funcs with extra parameters
# so CTC loss is implemented in a lambda layer
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

# clipnorm seems to speeds up convergence
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
# rms = RMSprop(lr=0.001, clipnorm=5, clipvalue=0.5)

In [0]:
# model = Model(inputs=input_data, outputs=y_pred)
model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

In [0]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 1595, 64, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 1595, 64, 64) 640         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 797, 32, 64)  0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 797, 32, 128) 73856       max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [24]:
from keras.models import load_model

print("Loading pre-trained weight")
model.load_weights('drive/My Drive/Machine_Learning-prj/vietnamese_address/weight/20xaugument-weights-training-improvement-05-82.40.hdf5')

Loading pre-trained weight


In [0]:
from keras.callbacks import ModelCheckpoint

filepath = 'drive/My Drive/Machine_Learning-prj/vietnamese_address/weight/3xaugument-weights-training-improvement-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
train_idx = 28000
test_idx = 28600
batch_size = 64

In [0]:
# model.fit([images, labels_script, x_train_len, y_train_len], np.zeros(len(images)), validation_split=0.01 ,batch_size=16, epochs=200, callbacks=callbacks_list)


In [27]:
model.fit_generator(generator=image_batch_generator(images_augumented[:train_idx], labels_script[:train_idx], x_train_len[:train_idx], y_train_len[:train_idx], batch_size), steps_per_epoch=(len(images_augumented[:train_idx])//batch_size) , epochs=200, verbose=1, callbacks=callbacks_list, 
                    validation_data=image_batch_generator(images_augumented[train_idx:], labels_script[train_idx:], x_train_len[train_idx:], y_train_len[train_idx:], batch_size), validation_steps=(len(images_augumented[train_idx:])//batch_size))

Epoch 1/200
437/437 [==============================] - 1414s 3s/step - loss: 67.3695 - val_loss: 71.1530

Epoch 00001: val_loss improved from inf to 71.15297, saving model to drive/My Drive/Machine_Learning-prj/vietnamese_address/weight/3xaugument-weights-training-improvement-01-71.15.hdf5
Epoch 2/200
437/437 [==============================] - 1401s 3s/step - loss: 64.1026 - val_loss: 78.3943

Epoch 00002: val_loss did not improve from 71.15297
Epoch 3/200
437/437 [==============================] - 1381s 3s/step - loss: 61.5572 - val_loss: 73.0867

Epoch 00003: val_loss did not improve from 71.15297
Epoch 4/200
437/437 [==============================] - 1388s 3s/step - loss: 59.5838 - val_loss: 72.1636

Epoch 00004: val_loss did not improve from 71.15297
Epoch 5/200
437/437 [==============================] - 1364s 3s/step - loss: 58.2314 - val_loss: 66.8427

Epoch 00005: val_loss improved from 71.15297 to 66.84275, saving model to drive/My Drive/Machine_Learning-prj/vietnamese_address/

KeyboardInterrupt: ignored

In [0]:
model.input

[<tf.Tensor 'the_input:0' shape=(?, 32, 256, 1) dtype=float32>,
 <tf.Tensor 'the_labels_1:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'input_length_1:0' shape=(?, 1) dtype=int64>,
 <tf.Tensor 'label_length_1:0' shape=(?, 1) dtype=int64>]

In [0]:
model.output

<tf.Tensor 'ctc_2/ExpandDims:0' shape=(?, 1) dtype=float32>

In [0]:
loss_out

<tf.Tensor 'ctc/ExpandDims:0' shape=(?, 1) dtype=float32>

In [0]:
model.save_weights('drive/My Drive/Machine_Learning-prj/sentence_hand_written/model_weight.hdf5')

In [32]:
np.where(labels_script < 0)

(array([], dtype=int64), array([], dtype=int64))